In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline 

In [3]:
from yaml import load as yaml_load
import os
import sys
import json
sys.path.append('../')
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import LongType
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler, VectorIndexer, StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import LongType, StringType, FloatType, IntegerType, DoubleType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
import re
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml import Pipeline
spark = SparkSession.builder.master("local[*]").appName("Data cleaning").getOrCreate()

In [4]:
from src.data.make_dataset import LoadDataframe

In [5]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    
    
def duration_day_func(x):
    """
    :return:
    """
    from astral import Astral
    city_name = 'Chicago'
    a = Astral()
    a.solar_depression = 'civil'
    city = a[city_name]
    sun = city.sun(date=x, local=True)
    return float((sun['sunset'] - sun['sunrise']).total_seconds())

extract_blok = udf(lambda x : re.findall(r"(\w+)$", x)[0], StringType())
isStreet = udf(lambda x :  1 if x in ['ST', 'St', 'st'] else 0, IntegerType())
isAV = udf(lambda x : 1 if x in ['Ave', 'AV', 'AVE'] else 0, IntegerType())
isBLVD = udf(lambda x : 1 if x in ['BLVD'] else 0, IntegerType())
isRD = udf(lambda x : 1 if x in ['RD'] else 0, IntegerType())
isPL = udf(lambda x : 1 if x in ['PL', 'pl'] else 0, IntegerType())
isBROADWAY = udf(lambda x : 1 if x in ['BROADWAY', 'Broadway'] else 0, IntegerType())
isPKWY = udf(lambda x : 1 if x in ['PKWY', 'Pkwy'] else 0, IntegerType())
duration_day_udf = udf(lambda x :   duration_day_func(x), FloatType())

config_file = "/home/ml/Documents/crimes_chigaco/config/config.yml"
config = _build_configuration(config_file)

In [6]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2014')

CPU times: user 876 µs, sys: 0 ns, total: 876 µs
Wall time: 940 µs


In [7]:
df_temp = obj_df_loaded.df_temperature()
df_sky  = obj_df_loaded.df_sky()
df_socio = obj_df_loaded.df_socio()

In [8]:
from pyspark.sql.functions import isnan, when, count, col, isnull

In [9]:
features_socio = ['pct_housing_crowded',
 'pct_households_below_poverty','pct_age16_unemployed',
 'pct_age25_no_highschool',
 'pct_not_working_age',
 'per_capita_income',
 'hardship_index']
for f in features_socio:
    df_socio = df_socio.withColumn(f, df_socio[f].cast(FloatType()))

In [10]:
df_crime = obj_df_loaded.df_crime()
df_crime_socio = df_crime.join(df_socio, ['community_area_number'], "inner")

/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py:1793: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [11]:
df_crime_socio = df_crime_socio.na.drop()

In [12]:
df_crime_socio = df_crime_socio.withColumn("block_extract", extract_blok(df_crime_socio.block))

In [13]:
df_crime_socio = (df_crime_socio.
                  withColumn("isStreet",isStreet(df_crime_socio.block_extract)).
                  withColumn("isAV", isAV(df_crime_socio.block_extract)).
                  withColumn("isBLVD", isBLVD(df_crime_socio.block_extract)).
                  withColumn("isRD", isRD(df_crime_socio.block_extract)).
                  withColumn("isPL", isPL(df_crime_socio.block_extract)).
                  withColumn("isBROADWAY", isBROADWAY(df_crime_socio.block_extract)).
                  withColumn("isPKWY", isPKWY(df_crime_socio.block_extract)))

In [14]:
df_crime_socio = df_crime_socio.withColumn('duree_day', duration_day_udf('date'))

In [15]:
df_crime_socio.printSchema()

root
 |-- community_area_number: string (nullable = true)
 |-- id: string (nullable = true)
 |-- cas_number: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- block: string (nullable = true)
 |-- iucr: string (nullable = true)
 |-- primary_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- arrest: string (nullable = true)
 |-- domestic: string (nullable = true)
 |-- beat: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- fbi_code: string (nullable = true)
 |-- x_coordinate: string (nullable = true)
 |-- y_coordinate: string (nullable = true)
 |-- year: string (nullable = true)
 |-- updated_on: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- location: string (nullable = true)
 |-- community_area_name: string (nullable = true)
 |-- pct_housing_crowded: float (nullable = tru

In [16]:
df_crime_socio = (
df_crime_socio.withColumn("month", func.month(func.col("date"))).
withColumn("year", func.year(func.col("date"))).
withColumn("day", func.dayofmonth(func.col("date"))).
withColumn("hour", func.hour(func.col("date"))).withColumn("minute", func.minute(func.col("date"))).
withColumn("dayofmonth", func.dayofmonth(func.col("date"))).   
withColumn("dayofyear", func.dayofyear(func.col("date"))).
withColumn("dayofweek", func.dayofweek(func.col("date")))
)

In [17]:
df_temp = df_temp.drop('datetime')

In [18]:
df_total = df_crime_socio.join(df_temp, ['year', 'month','day','hour'], how = "inner").\
                        join(df_sky, ['year', 'month','day','hour'], how = "inner")

In [19]:
df_total = df_total.na.drop()

In [20]:
df_total = df_total.withColumn('latitude', df_total['latitude'].cast(FloatType()))
df_total = df_total.withColumn('longitude', df_total['longitude'].cast(FloatType()))

In [21]:
list_of_crimes = config["List_of_crimes_prediction"]["with_merge_pred"]

In [22]:
df = df_total.where(col('primary_type').isin(list_of_crimes))

In [23]:
df = df.cache()

In [24]:
categoricalColumns = ['domestic']
numericCols = ['year', 'month', 'day', 'hour','minute', 'latitude',
               'longitude', 'isStreet', 'isAV',
               'isBLVD', 'isRD', 'isPL', 'isBROADWAY',
               'isPKWY', 'duree_day', 'minute','dayofmonth',
               'dayofyear','dayofweek','Temperature',
               'pct_housing_crowded',
               'pct_households_below_poverty',
               'pct_age16_unemployed',
               'pct_age25_no_highschool',
               'pct_not_working_age',
               'per_capita_income',
               'hardship_index',
                'Chicago_broken clouds',
                'Chicago_drizzle',
                 'Chicago_few clouds',
                'Chicago_fog',
                 'Chicago_haze',
                 'Chicago_heavy intensity drizzle',
                 'Chicago_heavy intensity rain',
                 'Chicago_heavy snow',
                 'Chicago_light intensity drizzle',
                 'Chicago_light rain',
                 'Chicago_light rain and snow',
                 'Chicago_light snow',
                 'Chicago_mist',
                 'Chicago_moderate rain',
                 'Chicago_overcast clouds',
                 'Chicago_proximity thunderstorm',
                 'Chicago_scattered clouds',
                 'Chicago_sky is clear',
                 'Chicago_snow',
                 'Chicago_thunderstorm',
                 'Chicago_thunderstorm with heavy rain',
                 'Chicago_thunderstorm with light rain',
                 'Chicago_thunderstorm with rain',
                'Chicago_very heavy rain'
]


In [25]:
rf = RandomForestClassifier(labelCol='label', featuresCol='features',numTrees=150,  maxDepth=8)

In [26]:
stages = []
label_stringIdx = StringIndexer(inputCol = 'primary_type', outputCol = 'label')
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages.append(stringIndexer)
    stages.append(encoder)
stages.append(label_stringIdx)
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages.append(assembler)
stages.append(rf)

In [27]:
df_train, df_test = df.randomSplit([0.7, 0.3])

In [28]:
pipeline_and_model = Pipeline(stages = stages)

In [ ]:
%%time
pipeline_model = pipeline_and_model.fit(df_train)

In [ ]:
%%time
df_test_pred = pipeline_model.transform(df_test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
predictions = df_test_pred.select(col("label"), col("prediction"))

In [ ]:
%%time
predictions_ = predictions.cache()

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")


In [ ]:
accuracy = evaluator.evaluate(predictions_)

In [ ]:
print(accuracy)